In [69]:
from openai import OpenAI
import os
import json
import pandas as pd
from IPython.display import display_markdown

client = OpenAI()

# Set parameters

name = "1"
model = "gpt-3.5-turbo"               # important options: "gpt-4o-mini", "gpt-4o", "gpt-3.5-turbo"
loops = 1
memory = 1
stable_system_message = False
assistant_transform = False    
assistant_transform_type = "user"     # options: "user", "user-assistant", "system"
stream = True

first_prompt = [
  #{"role": "system", "content": ""},
  {"role": "user", "content": ""}
]

# Choose or create the right folder with json files

folder_name = f"{model} memory-{memory}"
if assistant_transform:
  folder_name += f" {assistant_transform_type}"
os.makedirs(folder_name, exist_ok=True)

for loop in range(loops):

  # Load completions from json, if continuing with a file generated in the past

  try:
    with open(f"{folder_name}/{name}.json", "r") as json_file:
      completions=json.load(json_file)

  # If not, create a new completions database

  except FileNotFoundError:
    completions = []

  # Collect messages generated so far

  messages = first_prompt
  for completion in completions:
      message = completion["choices"][0]["message"]
      messages.append({"role": message["role"], "content": message["content"]}   
      )

  # Pick prompt messages

  if memory >= len(messages):
      prompt_messages = messages
  else:
      #print(f"Memory smaller than the entire conversation. Forgetting {len(messages)-memory} messages")
      prompt_messages = []
      if stable_system_message:
        prompt_messages.append(first_prompt[0])
        print(prompt_messages)
        print("stable_system_message")
      prompt_messages = prompt_messages+messages[(len(messages)-memory):]

  # Transform assistant role to assistant and user, user, or system roles

  if assistant_transform:
    if assistant_transform_type == "user-assistant":
      i = 1
      while i <= len(prompt_messages):
        if prompt_messages[-i]["role"] == "assistant":
          prompt_messages[-i]["role"] = "user"
        i += 2
    else:
      for prompt_message in prompt_messages:
        if prompt_message["role"] == "assistant":
          prompt_message["role"] = assistant_transform_type


  # Send to OpenAI API to get a new completion

  completion = client.chat.completions.create(
        model=model,
        messages = prompt_messages,
        stream = stream,
      )
  print(f"Prompt messages sent to OpenAI API:{prompt_messages}")
  if stream:
    all_completion_data = []
    content_string = ""
    for chunk in completion:
      whole_completion.append(chunk)
      print(chunk.choices[0].delta.content or "", end="")
      content += chunk.choices[0].delta.content
    completion = all_completion_data[0]
    completion.choices[0].delta.content = content
    normalized_choices = pd.json_normalize(completion, record_path=['choices','delta'], record_prefix='')
    completion.choices = normalized_choices
  else:
    print(completion.choices[0].message.content)

  # Update database and save it to the right json file

  completions.append(completion.model_dump())
  with open(f"{folder_name}/{name}.json", "w") as json_file:
      json.dump(completions,json_file)

 

Prompt messages sent to OpenAI API:[{'role': 'assistant', 'content': 'Hello! How can I help you today?'}]
Hello! How can I assist you today?

AttributeError: 'Stream' object has no attribute 'model_dump'

In [70]:
whole_completion

[ChatCompletionChunk(id='chatcmpl-9rVqFsodmLisXKz5pYW67GSXOsexL', choices=[Choice(delta=ChoiceDelta(content='', function_call=None, role='assistant', tool_calls=None), finish_reason=None, index=0, logprobs=None)], created=1722540811, model='gpt-3.5-turbo-0125', object='chat.completion.chunk', service_tier=None, system_fingerprint=None, usage=None),
 ChatCompletionChunk(id='chatcmpl-9rVqFsodmLisXKz5pYW67GSXOsexL', choices=[Choice(delta=ChoiceDelta(content='Hello', function_call=None, role=None, tool_calls=None), finish_reason=None, index=0, logprobs=None)], created=1722540811, model='gpt-3.5-turbo-0125', object='chat.completion.chunk', service_tier=None, system_fingerprint=None, usage=None),
 ChatCompletionChunk(id='chatcmpl-9rVqFsodmLisXKz5pYW67GSXOsexL', choices=[Choice(delta=ChoiceDelta(content='!', function_call=None, role=None, tool_calls=None), finish_reason=None, index=0, logprobs=None)], created=1722540811, model='gpt-3.5-turbo-0125', object='chat.completion.chunk', service_tier

In [68]:
print(completion)

In [66]:
print(dir(completion))

['__annotations__', '__class__', '__class_getitem__', '__delattr__', '__dict__', '__dir__', '__doc__', '__enter__', '__eq__', '__exit__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__next__', '__orig_bases__', '__orig_class__', '__parameters__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__stream__', '__subclasshook__', '__weakref__', '_cast_to', '_client', '_decoder', '_iter_events', '_iterator', 'close', 'response']


In [55]:
## Display data

# Load the JSON file

file_path = f"{folder_name}/{name}.json"

#file_path = f"gpt-3.5-turbo memory-1 user/2.json"

with open(file_path, 'r') as file:
    data = json.load(file)

# Normalize the JSON data in completions

normalized_data = pd.json_normalize(data, record_path='choices', meta=[
    'id', 'created', 'model', 'object', 'service_tier', 'system_fingerprint', 
    ['usage', 'completion_tokens'], ['usage', 'prompt_tokens'], ['usage', 'total_tokens']], sep='.',
    meta_prefix='', record_prefix='choices.')

# Make more important columns more visible

normalized_data = normalized_data[['choices.message.content', 'model', 
       'usage.completion_tokens', 'usage.prompt_tokens', 'usage.total_tokens', 'id', 
       'choices.finish_reason', 'choices.index', 'choices.logprobs', 
       'choices.message.role', 'choices.message.function_call', 'choices.message.tool_calls', 
       'created', 'object', 'service_tier', 'system_fingerprint'
       ]]

# Display

content_all = normalized_data["choices.message.content"]
last = content_all.iloc[-1]

print(file_path)
print(len(content_all))

for content in content_all:
   display_markdown(content, raw=True)

gpt-3.5-turbo memory-1/1.json
5


Hello! How can I assist you today?

Hello! How can I assist you today?

Hello! How can I assist you today?

Hello! How can I assist you today?

Hello! How can I help you today?

In [44]:
completions[4]

{'id': 'chatcmpl-9rSyNew3bRbcf5FoSt88NJ9iu4ljW',
 'choices': [{'finish_reason': 'stop',
   'index': 0,
   'logprobs': None,
   'message': {'content': 'Some categories I can assist with include (but are not limited to):\n- General knowledge\n- Science\n- Technology\n- History\n- Health\n- Literature\n- Geography\n- Mathematics\n- Language and linguistics\n\nFeel free to ask me anything within these categories or any other topics you may be curious about!',
    'role': 'assistant',
    'function_call': None,
    'tool_calls': None}}],
 'created': 1722529783,
 'model': 'gpt-3.5-turbo-0125',
 'object': 'chat.completion',
 'service_tier': None,
 'system_fingerprint': None,
 'usage': {'completion_tokens': 64, 'prompt_tokens': 43, 'total_tokens': 107}}